In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import constants

# Get symbols from constants file
lines = constants.sa_str.splitlines()
symbols = [line.split("\t")[1] for line in lines][:100]

# Download data
df = yf.download(symbols + ["SPY"], period="1y", interval="1h", ignore_tz=True)


[*********************100%***********************]  101 of 101 completed


In [15]:
close = df["Close"]
# Check NaN ratios
for sym in symbols:
    sym_close = df["Close"][sym].to_numpy()
    nan_ratio = np.isnan(sym_close).sum() / len(sym_close)
    if nan_ratio > 0.1:
        print(f"{sym} nan ratio: {nan_ratio:.4f} - dropping")
        close = close.drop(sym, axis=1)

# Convert to np array
close_np = close.to_numpy()
close_np.shape


(1747, 101)

In [43]:
x, y = [], []
for i in range(len(df.index)):
    if len(df.index) - i > 6:
        c0 = df.index[i].hour == 9
        c1 = df.index[i+3].hour == 12
        c2 = df.index[i+6].hour == 15
        if c0 and c1 and c2:
            x.append(close_np[i+3] / close_np[i] - 1)
            y.append(close_np[i+6] / close_np[i+3] - 1)

x_np, y_np = np.stack(x), np.stack(y)
x_std = (x_np - x_np.mean()) / x_np.std() / 2
y_std = (y_np - y_np.mean()) / y_np.std() / 2

In [44]:
x_std.std()

np.float64(0.5)

spirit = 150


In [ ]:
days = []
for i in range(len(df.index)):
    if df.index[i].hour == 9 and len(df.index) - i > 6:
        day_data = []
        for sym in symbols:
            vol_0 = df["Volume"][sym][df.index[i]]

            vol_1 = (vol_0 + 
                df["Volume"][sym][df.index[i + 1]] +
                df["Volume"][sym][df.index[i + 2]] + 
                df["Volume"][sym][df.index[i + 3]])
            
            vol_2 = (vol_1 + 
                df["Volume"][sym][df.index[i + 4]] +
                df["Volume"][sym][df.index[i + 5]] + 
                df["Volume"][sym][df.index[i + 6]])
            
            sym_data = [
                df["Close"][sym][df.index[i]],
                df["Close"][sym][df.index[i + 3]],
                df["Close"][sym][df.index[i + 6]],
                vol_0,
                vol_1,
                vol_2,
            ]

            day_data.append(sym_data)

        days.append(day_data)
# days_np = np.concatenate(days)